**If you found this notebook helpful please upvote and comment, it really helps me out!**

# **Importing Libraries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from random import choices

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score

RS = 69420
tf.random.set_seed(RS)
np.random.seed(RS)

# **Loading Data**

**You can see we only use values after day 85, this is due to all the excellent exploration notebooks which have shown JaneSt to recalibrate models post day 85**

**I am using a HDF5 version of the training data because it loads nearly 10x faster than csv's, let me know if you would like access**

In [ ]:
train = pd.read_hdf("../input/janesthdf/train.hdf5")
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

In [ ]:
train['action'] = ((train['resp'].values) > 0).astype(int)
features = [c for c in train.columns if "feature" in c]
f_mean = np.mean(train[features[1:]].values,axis=0)
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

In [ ]:
X = train.loc[:, train.columns.str.contains('feature')]
y = train['action']

# **Train-Test-Validation**

In [ ]:
from sklearn.model_selection import train_test_split
# 60, 20, 20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RS)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RS)

In [ ]:
from sklearn.impute import SimpleImputer
impute = SimpleImputer(missing_values = np.nan)

X_train = impute.fit_transform(X_train)
X_test = impute.transform(X_test)
X_val = impute.transform(X_val)

# **Building the   *W I D E*   boy**

**Use callbacks because we hate overfitting**

In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [ ]:
def create_mlp(num_columns, label_smoothing):
    
    model = Sequential()
    model.add(Input(shape=(num_columns,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Dense(1280))
    model.add(BatchNormalization())
    model.add(Activation(tf.keras.activations.swish))
    model.add(Dropout(0.3))
    
    model.add(Dense(1280))
    model.add(BatchNormalization())
    model.add(Activation(tf.keras.activations.swish))
    model.add(Dropout(0.3))
    
    model.add(Dense(1280))
    model.add(BatchNormalization())
    model.add(Activation(tf.keras.activations.swish))
    model.add(Dropout(0.3))

    model.add(Dense(1))
    model.add(Activation("sigmoid"))
    
    model.compile(
                    optimizer = 'adam',
                    loss = BinaryCrossentropy(label_smoothing = label_smoothing),
                    metrics= [AUC(), 'acc']
                 )

    return model

In [ ]:
label_smoothing = 1e-2

# clf = create_mlp(len(features), label_smoothing)

In [ ]:
# clf.summary()

# **Train dat bih**

In [ ]:
batch_size = 2560

*More Robust application of this model as it utilizes early stopping upon the validation set we split out*

In [ ]:
# history = clf.fit(X_train, y_train, validation_data = (X_val, y_val), callbacks = callback, epochs=1000, batch_size = batch_size)

* *Tbh I dont really know why I chose to train a mega-wide network, largely its for the memes and so I can say   W I D E B O I*  
* I would appreciate any academic rigour all you wonderful kagglers could provide!

In [ ]:
# history = pd.DataFrame(clf.history.history)
# history[['val_loss', 'loss']].plot()

# **Results**

Remember that ***Accuracy is not everything*** this is a markets related competition, the markets are so dynamic there is no need for extreme accuracy, rather we want to focus on the gerneralisability of the model

In [ ]:
# y_pred = clf.predict_proba(X_test)
# y_valpred = clf.predict(X_val)

In [ ]:
# If you have any recommendations to tidy this code up please comment below

# th = 0.5
# y_preds = []
# y_valpreds = []

# for i in y_pred:
#     if i < 0.5:
#         y_preds.append(0)
#     else:
#         y_preds.append(1)
    
# for i in y_valpred:
#     if i < 0.5:
#         y_valpreds.append(0)
#     else:
#         y_valpreds.append(1)

In [ ]:
# mean_acc = np.mean((accuracy_score(y_test, y_preds),
#                     accuracy_score(y_val, y_valpreds)))

# print("The Testing Accuracy is: {}".format(accuracy_score(y_test, y_preds)))
# print("The Validation Accuracy is: {}".format(accuracy_score(y_val, y_valpreds)))
# print("The Mean Accuracy is: {}".format(mean_acc))

In [ ]:
# clf.save('wide_boi.hdf5')

# **Submission**

**To prepare this notebook for submission first actually run and train the model (all lines above) then save and download the model. Comment out all the code above when ready to submit and then after this you will be able to re-upload the model to kaggle and load it in again below for submission**

In [ ]:
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import pandas as pd

clf = tf.keras.models.load_model("../input/wide-boi-3/wide_boi.hdf5")

f_mean = 0.2674099
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

# **Submitting Predictions**

In [ ]:
# Submission Code Thanks to: https://www.kaggle.com/tarlannazarov/own-jane-street-with-keras-nn

models = []

models.append(clf)

th = 0.5000


f = np.median
models = models[-3:]
import janestreet
env = janestreet.make_env()

for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

**If you found this notebook helpful please upvote and comment, it really helps me out!**